In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from scipy import ndimage
from keras.optimizers import Adadelta,Adam
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint
# from lipnet.lipreading.generators import BasicGenerator
# from lipnet.lipreading.callbacks import Statistics, Visualize
# from lipnet.lipreading.curriculums import Curriculum
# from lipnet.core.decoders import Decoder
# from lipnet.lipreading.helpers import labels_to_text
# from lipnet.utils.spell import Spell
from lipnet.model2 import LipNet
import numpy as np
import datetime
import cv2
import skvideo.io
import dlib
from scipy.misc import imresize
# from lipnet.lipreading.aligns import Align
import imageio
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
import random
import tensorflow as tf
import keras.backend as K
tf_config = tf.ConfigProto(allow_soft_placement=False)
tf_config.gpu_options.allow_growth = True
s = tf.Session(config=tf_config)
K.set_session(s)

FACE_PREDICTOR_PATH  = os.path.join('common','predictors','shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(FACE_PREDICTOR_PATH)

frames_count = 30
words_folder = 'training/unseen_speakers/TARGET/datasets/lipread_mp4'
words = os.listdir(words_folder)
words = ['EVERYONE',
 'EVENTS',
 'EVERYBODY',
 'CLOUD',
 'RIGHTS',
 'AUTHORITIES',
 'WITHIN',
 'SEVERAL',
 'ABSOLUTELY',
 'WINDS',
 'AFTERNOON',
 'BETWEEN',
 'TALKING',
 'AGREEMENT',
 'PRICES'] #words[:2]
words = [s.upper() for s in words]
max_len = 12 #np.max([len(word) for word in words])
used_words = {''}
print(words)
def datagen(data_partition = 'train', batch_size = 16):
    while True:
        counter = 0
        word_idx = np.random.randint(len(words))
        used_words.add(words[word_idx])
        train_folder = 'training/unseen_speakers/TARGET/datasets/lipread_mp4/{0}/{1}/'.format(words[word_idx],data_partition)
    #         print(train_folder)
        files = os.listdir(train_folder)
#         print(files)
        paths = list(set([x.split('.')[0] for x in files]))
        
        X_data = []#np.zeros((batch_size,))
        Y_data = []
        input_length = []
        label_length = []
        source_str = []
        while counter<batch_size:
#             print(counter)
            idx = np.random.randint(len(paths))
            path = os.path.join(train_folder,paths[idx])
            frames = []
            for img_name in sorted(os.listdir(path)):
                img = cv2.imread(os.path.join(path,img_name))
                frames.append(img[...,::-1])
            frames = np.array(frames)
            if len(frames.shape)==4:
                frames = np.rollaxis(frames,2, start=1)
    #             print(path)
    #             print(frames.shape)
    #             print('---------')
                if 'numpy.ndarray' in str(type(frames)) \
                        and len(np.shape(frames))==4 \
                        and np.shape(frames)[1] == 100 \
                        and np.shape(frames)[2] == 50:

                    X_data.append(frames)


                    ####
                    Y_data.append([ord(c) - 65 for c in words[word_idx]]+[-1]*(max_len-len(words[word_idx])))
                    input_length.append(frames_count)
                    label_length.append(len(words[word_idx]))
                    source_str.append(words[word_idx])
                    counter+=1
                else:
                    continue
            else:
                print('error:', frames.shape, path)
                continue

        X_data = np.array(X_data).astype(np.float32) / 255 
        Y_data = np.array(Y_data)
        input_length = np.array(input_length)
        label_length = np.array(label_length)
        source_str = np.array(source_str)
        
        inputs = {'the_input': X_data,
                  'the_labels': Y_data,
                  'input_length': input_length,
                  'label_length': label_length,
                  'source_str': source_str  # used for visualization only
                  }
        outputs = {'ctc': np.zeros([batch_size])}
        
#         print('the_input:', inputs['the_input'].shape)
#         print('the_labels:', inputs['the_labels'].shape)
#         print('input_length:', inputs['input_length'].shape)
#         print('label_length:', inputs['label_length'].shape)
#         print('source_str:', inputs['source_str'].shape)
#         print('batch_frames_shape',X_data.shape, data_partition)
        yield (inputs, outputs)

['EVERYONE', 'EVENTS', 'EVERYBODY', 'CLOUD', 'RIGHTS', 'AUTHORITIES', 'WITHIN', 'SEVERAL', 'ABSOLUTELY', 'WINDS', 'AFTERNOON', 'BETWEEN', 'TALKING', 'AGREEMENT', 'PRICES']


In [3]:
# global lip_gen

run_name = datetime.datetime.now().strftime('%Y:%m:%d_%H:%M:%S')
run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size = \
    run_name, 0, 1000, 3, 100, 50, 30, 12, 2



CURRENT_PATH = os.getcwd()+'/training/unseen_speakers'
DATASET_DIR  = os.path.join(CURRENT_PATH, 'datasets')
OUTPUT_DIR   = os.path.join(CURRENT_PATH, 'results')
LOG_DIR      = os.path.join(CURRENT_PATH, 'logs')
# if not os.path.exists('training/unseen_speakers/results/'+run_name):
#     os.makedirs('training/unseen_speakers/results/'+run_name)
if not os.path.exists('training/unseen_speakers/logs/'+run_name):
    os.makedirs('training/unseen_speakers/logs/'+run_name)

PREDICT_GREEDY      = False
PREDICT_BEAM_WIDTH  = 200
PREDICT_DICTIONARY  = os.path.join(CURRENT_PATH,'..','..','common','dictionaries','lfw.txt')
FACE_PREDICTOR_PATH  = os.path.join(CURRENT_PATH,'..','..','common','predictors','shape_predictor_68_face_landmarks.dat')

# def curriculum_rules(epoch):
#     return { 'sentence_length': -1, 'flip_probability': 0.5, 'jitter_probability': 0.05 }


# # def train(run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size):
# curriculum = Curriculum(curriculum_rules)
# lip_gen = BasicGenerator(dataset_path=DATASET_DIR, #vtype = "face", face_predictor_path = FACE_PREDICTOR_PATH,
#                             minibatch_size=minibatch_size,
#                             img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
#                             absolute_max_string_len=absolute_max_string_len,
#                             curriculum=curriculum, start_epoch=start_epoch).build()
# lip_gen_train = datagen(data_partition='train', batch_size=5)
# lip_gen_val = datagen(data_partition='val', batch_size=5)

lipnet = LipNet(img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
                        absolute_max_string_len=absolute_max_string_len, output_size=27)
# lipnet.summary()
# adadelta = Adadelta(lr=0.0001, decay=1e-08, epsilon=1e-08)

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=1e-08)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam, metrics=['accuracy'])


# spell = Spell(path=PREDICT_DICTIONARY)
# decoder = Decoder(greedy=PREDICT_GREEDY, beam_width=PREDICT_BEAM_WIDTH,
#                   postprocessors=[labels_to_text, spell.sentence])

# define callbacks
# statistics  = Statistics(lipnet, lip_gen.next_val(), decoder, 256, output_dir=os.path.join(OUTPUT_DIR, run_name))
# visualize   = Visualize(os.path.join(OUTPUT_DIR, run_name), lipnet, lip_gen.next_val(), decoder, num_display_sentences=minibatch_size)
tensorboard = TensorBoard(log_dir=os.path.join(LOG_DIR, 'crop-'+run_name))
# csv_logger  = CSVLogger(os.path.join(LOG_DIR, "{}-{}.csv".format('training',run_name)), separator=',', append=True)
checkpoint  = ModelCheckpoint(save_best_only=True,
                              filepath='training/unseen_speakers/results/weights_lfw-crop.hdf5'
                              , monitor='val_loss'
                              , verbose=1
                              , mode='min'
                              , period=1)
    
    # x = next(lip_gen.next_val())
    # for i in x:
    #     print('$$$$$----------', type(i))





y_pred (?, 30, 27)


In [4]:
# run_name = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S')
# train(run_name, 0, 1000, 3, 100, 50, 75, 32, 2)

In [5]:
# os.getcwd()+'training/unseen_speakers'

In [6]:
# g=lip_gen.next_train()
# inputs, outputs = next(g)

In [7]:
# inputs

In [8]:
# len('place green at t six now')

In [9]:
# len([ 15.,  11.,   0.,   2.,   4.,  26.,   6.,  17.,   4.,   4.,  13.,
#           26.,   0.,  19.,  26.,  19.,  26.,  18.,   8.,  23.,  26.,  13.,
#           14.,  22.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.])

In [10]:
# words

In [ ]:
# load weight if necessary

start_epoch = 85
if start_epoch > 0:
    weight_file = os.path.join(OUTPUT_DIR, 'weights_lfw-crop.hdf5')
    lipnet.model.load_weights(weight_file)

lipnet.model.fit_generator(generator=datagen(data_partition='train', batch_size=32),
                    steps_per_epoch=1
                    , epochs=50000,
                    #steps_per_epoch=lip_gen.default_training_steps, epochs=stop_epoch,
                    validation_data=datagen(data_partition='val', batch_size=32)
                    , validation_steps=1,#validation_steps=lip_gen.default_validation_steps,
                    callbacks=[checkpoint,tensorboard],                         
                    initial_epoch=start_epoch, 
                    verbose=1)
                    #max_q_size=5,
                    #workers=2,
                    #pickle_safe=True)

Epoch 86/50000
1/1 [==============================] - 4s 4s/step - loss: 16.6640 - acc: 0.0000e+00 - val_loss: 29.2938 - val_acc: 0.0000e+00

Epoch 00086: val_loss improved from inf to 29.29377, saving model to training/unseen_speakers/results/weights_lfw-crop.hdf5
Epoch 87/50000
1/1 [==============================] - 0s 308ms/step - loss: 20.3870 - acc: 0.0000e+00 - val_loss: 24.2630 - val_acc: 0.0000e+00

Epoch 00087: val_loss improved from 29.29377 to 24.26301, saving model to training/unseen_speakers/results/weights_lfw-crop.hdf5
Epoch 88/50000
1/1 [==============================] - 0s 306ms/step - loss: 14.8257 - acc: 0.0000e+00 - val_loss: 38.4674 - val_acc: 0.0000e+00

Epoch 00088: val_loss did not improve from 24.26301
Epoch 89/50000
1/1 [==============================] - 0s 334ms/step - loss: 24.9116 - acc: 0.0000e+00 - val_loss: 25.9389 - val_acc: 0.0000e+00

Epoch 00089: val_loss did not improve from 24.26301
Epoch 90/50000
error: (30,) training/unseen_speakers/TARGET/datase


Epoch 00124: val_loss did not improve from 13.61104
Epoch 125/50000
1/1 [==============================] - 0s 320ms/step - loss: 18.9958 - acc: 0.0000e+00 - val_loss: 25.5839 - val_acc: 0.0000e+00

Epoch 00125: val_loss did not improve from 13.61104
Epoch 126/50000
1/1 [==============================] - 0s 307ms/step - loss: 22.4435 - acc: 0.0000e+00 - val_loss: 26.1411 - val_acc: 0.0000e+00

Epoch 00126: val_loss did not improve from 13.61104
Epoch 127/50000
1/1 [==============================] - 0s 322ms/step - loss: 26.2070 - acc: 0.0000e+00 - val_loss: 18.3980 - val_acc: 0.0000e+00

Epoch 00127: val_loss did not improve from 13.61104
Epoch 128/50000
1/1 [==============================] - 0s 313ms/step - loss: 22.0106 - acc: 0.0000e+00 - val_loss: 19.1870 - val_acc: 0.0000e+00

Epoch 00128: val_loss did not improve from 13.61104
Epoch 129/50000
1/1 [==============================] - 0s 305ms/step - loss: 23.6603 - acc: 0.0000e+00 - val_loss: 18.2013 - val_acc: 0.0000e+00

Epoch 001


Epoch 00164: val_loss did not improve from 11.54269
Epoch 165/50000
1/1 [==============================] - 0s 307ms/step - loss: 25.4730 - acc: 0.0000e+00 - val_loss: 23.5508 - val_acc: 0.0000e+00

Epoch 00165: val_loss did not improve from 11.54269
Epoch 166/50000
1/1 [==============================] - 0s 319ms/step - loss: 21.3047 - acc: 0.0000e+00 - val_loss: 27.4709 - val_acc: 0.0000e+00

Epoch 00166: val_loss did not improve from 11.54269
Epoch 167/50000
1/1 [==============================] - 0s 307ms/step - loss: 18.1984 - acc: 0.0000e+00 - val_loss: 29.8686 - val_acc: 0.0000e+00

Epoch 00167: val_loss did not improve from 11.54269
Epoch 168/50000
1/1 [==============================] - 0s 320ms/step - loss: 25.1266 - acc: 0.0000e+00 - val_loss: 29.3326 - val_acc: 0.0000e+00

Epoch 00168: val_loss did not improve from 11.54269
Epoch 169/50000
1/1 [==============================] - 0s 303ms/step - loss: 20.6286 - acc: 0.0000e+00 - val_loss: 24.0230 - val_acc: 0.0000e+00

Epoch 001


Epoch 00206: val_loss did not improve from 11.54269
Epoch 207/50000
1/1 [==============================] - 0s 309ms/step - loss: 13.4416 - acc: 0.0000e+00 - val_loss: 22.5795 - val_acc: 0.0000e+00

Epoch 00207: val_loss did not improve from 11.54269
Epoch 208/50000
1/1 [==============================] - 0s 320ms/step - loss: 35.1728 - acc: 0.0000e+00 - val_loss: 20.0411 - val_acc: 0.0000e+00

Epoch 00208: val_loss did not improve from 11.54269
Epoch 209/50000
1/1 [==============================] - 0s 353ms/step - loss: 17.3244 - acc: 0.0000e+00 - val_loss: 25.3466 - val_acc: 0.0000e+00

Epoch 00209: val_loss did not improve from 11.54269
Epoch 210/50000
1/1 [==============================] - 0s 302ms/step - loss: 12.9838 - acc: 0.0000e+00 - val_loss: 18.0270 - val_acc: 0.0000e+00

Epoch 00210: val_loss did not improve from 11.54269
Epoch 211/50000
1/1 [==============================] - 0s 310ms/step - loss: 12.3147 - acc: 0.0000e+00 - val_loss: 10.8923 - val_acc: 0.0000e+00

Epoch 002


Epoch 00246: val_loss did not improve from 8.61113
Epoch 247/50000
1/1 [==============================] - 0s 306ms/step - loss: 17.2831 - acc: 0.0000e+00 - val_loss: 13.7296 - val_acc: 0.0000e+00

Epoch 00247: val_loss did not improve from 8.61113
Epoch 248/50000
1/1 [==============================] - 0s 311ms/step - loss: 19.2891 - acc: 0.0000e+00 - val_loss: 18.2289 - val_acc: 0.0000e+00

Epoch 00248: val_loss did not improve from 8.61113
Epoch 249/50000
1/1 [==============================] - 0s 304ms/step - loss: 18.8049 - acc: 0.0000e+00 - val_loss: 17.2423 - val_acc: 0.0000e+00

Epoch 00249: val_loss did not improve from 8.61113
Epoch 250/50000
1/1 [==============================] - 0s 307ms/step - loss: 23.2188 - acc: 0.0000e+00 - val_loss: 13.8213 - val_acc: 0.0000e+00

Epoch 00250: val_loss did not improve from 8.61113
Epoch 251/50000
1/1 [==============================] - 0s 307ms/step - loss: 19.6522 - acc: 0.0000e+00 - val_loss: 13.9972 - val_acc: 0.0000e+00

Epoch 00251: v

...


Epoch 01574: val_loss did not improve from 8.61113
Epoch 1575/50000
1/1 [==============================] - 0s 312ms/step - loss: 10.2901 - acc: 0.0000e+00 - val_loss: 21.4509 - val_acc: 0.0000e+00

Epoch 01575: val_loss did not improve from 8.61113
Epoch 1576/50000
1/1 [==============================] - 0s 313ms/step - loss: 21.8509 - acc: 0.0000e+00 - val_loss: 21.1733 - val_acc: 0.0000e+00

Epoch 01576: val_loss did not improve from 8.61113
Epoch 1577/50000
1/1 [==============================] - 0s 318ms/step - loss: 15.9165 - acc: 0.0000e+00 - val_loss: 14.9392 - val_acc: 0.0000e+00

Epoch 01577: val_loss did not improve from 8.61113
Epoch 1578/50000
1/1 [==============================] - 0s 315ms/step - loss: 18.5989 - acc: 0.0000e+00 - val_loss: 9.7703 - val_acc: 0.0000e+00

Epoch 01578: val_loss did not improve from 8.61113
Epoch 1579/50000
1/1 [==============================] - 0s 323ms/step - loss: 13.8880 - acc: 0.0000e+00 - val_loss: 21.3899 - val_acc: 0.0000e+00

Epoch 0157


Epoch 01616: val_loss did not improve from 8.61113
Epoch 1617/50000
1/1 [==============================] - 0s 307ms/step - loss: 16.0112 - acc: 0.0000e+00 - val_loss: 29.5620 - val_acc: 0.0000e+00

Epoch 01617: val_loss did not improve from 8.61113
Epoch 1618/50000
1/1 [==============================] - 0s 321ms/step - loss: 13.9030 - acc: 0.0000e+00 - val_loss: 29.5229 - val_acc: 0.0000e+00

Epoch 01618: val_loss did not improve from 8.61113
Epoch 1619/50000
1/1 [==============================] - 0s 309ms/step - loss: 12.0248 - acc: 0.0000e+00 - val_loss: 10.9957 - val_acc: 0.0000e+00

Epoch 01619: val_loss did not improve from 8.61113
Epoch 1620/50000
1/1 [==============================] - 0s 309ms/step - loss: 21.9232 - acc: 0.0000e+00 - val_loss: 10.8926 - val_acc: 0.0000e+00

Epoch 01620: val_loss did not improve from 8.61113
Epoch 1621/50000
1/1 [==============================] - 0s 306ms/step - loss: 18.6167 - acc: 0.0000e+00 - val_loss: 22.8634 - val_acc: 0.0000e+00

Epoch 016


Epoch 01658: val_loss did not improve from 8.61113
Epoch 1659/50000
1/1 [==============================] - 0s 310ms/step - loss: 21.6500 - acc: 0.0000e+00 - val_loss: 15.0715 - val_acc: 0.0000e+00

Epoch 01659: val_loss did not improve from 8.61113
Epoch 1660/50000
1/1 [==============================] - 0s 311ms/step - loss: 10.4505 - acc: 0.0000e+00 - val_loss: 15.0325 - val_acc: 0.0000e+00

Epoch 01660: val_loss did not improve from 8.61113
Epoch 1661/50000
1/1 [==============================] - 0s 324ms/step - loss: 20.9325 - acc: 0.0000e+00 - val_loss: 13.6343 - val_acc: 0.0000e+00

Epoch 01661: val_loss did not improve from 8.61113
Epoch 1662/50000
1/1 [==============================] - 0s 317ms/step - loss: 18.9558 - acc: 0.0000e+00 - val_loss: 15.1702 - val_acc: 0.0000e+00

Epoch 01662: val_loss did not improve from 8.61113
Epoch 1663/50000
1/1 [==============================] - 0s 312ms/step - loss: 13.7949 - acc: 0.0000e+00 - val_loss: 18.8031 - val_acc: 0.0000e+00

Epoch 016


Epoch 01700: val_loss did not improve from 8.61113
Epoch 1701/50000
1/1 [==============================] - 0s 306ms/step - loss: 14.4903 - acc: 0.0000e+00 - val_loss: 11.5605 - val_acc: 0.0000e+00

Epoch 01701: val_loss did not improve from 8.61113
Epoch 1702/50000
1/1 [==============================] - 0s 311ms/step - loss: 14.6024 - acc: 0.0000e+00 - val_loss: 14.9147 - val_acc: 0.0000e+00

Epoch 01702: val_loss did not improve from 8.61113
Epoch 1703/50000
1/1 [==============================] - 0s 314ms/step - loss: 11.8232 - acc: 0.0000e+00 - val_loss: 10.3428 - val_acc: 0.0000e+00

Epoch 01703: val_loss did not improve from 8.61113
Epoch 1704/50000
1/1 [==============================] - 0s 311ms/step - loss: 14.1123 - acc: 0.0000e+00 - val_loss: 11.0917 - val_acc: 0.0000e+00

Epoch 01704: val_loss did not improve from 8.61113
Epoch 1705/50000
1/1 [==============================] - 0s 309ms/step - loss: 17.3023 - acc: 0.0000e+00 - val_loss: 14.5056 - val_acc: 0.0000e+00

Epoch 017


Epoch 01741: val_loss did not improve from 8.61113
Epoch 1742/50000
1/1 [==============================] - 0s 316ms/step - loss: 15.5323 - acc: 0.0000e+00 - val_loss: 18.1765 - val_acc: 0.0000e+00

Epoch 01742: val_loss did not improve from 8.61113
Epoch 1743/50000
1/1 [==============================] - 0s 305ms/step - loss: 12.5619 - acc: 0.0000e+00 - val_loss: 9.5145 - val_acc: 0.0000e+00

Epoch 01743: val_loss did not improve from 8.61113
Epoch 1744/50000
1/1 [==============================] - 0s 330ms/step - loss: 13.8369 - acc: 0.0000e+00 - val_loss: 18.2277 - val_acc: 0.0000e+00

Epoch 01744: val_loss did not improve from 8.61113
Epoch 1745/50000
1/1 [==============================] - 0s 322ms/step - loss: 19.5099 - acc: 0.0000e+00 - val_loss: 12.6932 - val_acc: 0.0000e+00

Epoch 01745: val_loss did not improve from 8.61113
Epoch 1746/50000
1/1 [==============================] - 0s 310ms/step - loss: 12.1674 - acc: 0.0000e+00 - val_loss: 17.0051 - val_acc: 0.0000e+00

Epoch 0174

In [ ]:
# g=datagen(data_partition='test', batch_size=1)
# x,y = next(g)

In [ ]:
# used_words

In [ ]:
# x['the_input'][0].shape, x['source_str'], x['the_labels']


In [ ]:
# # im = batch_frames[1][25]
# import matplotlib.pyplot as plt
# %matplotlib inline
# # generator = datagen(min_row_number=0,max_row_number=200000, batch_size= 6)
# # X,Y = next(generator)
# X = x['the_input'][0]
# # print(X.shape, Y.shape)
# def plot_data(x,ax):
# #     x = x.reshape((size,size))
#     ax.imshow(x, cmap='gray')
# #     if y is not None:
# #         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
# fig = plt.figure(figsize=(30,30))
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
# for i in range(30):
#     ax = fig.add_subplot(1, 30, i + 1, xticks=[], yticks=[])
#     plot_data(X[i], ax)

In [ ]:
# # import io
# # import base64
# # from IPython.display import HTML

# # video = io.open('evaluation/samples/test.mp4', 'r+b').read()
# # encoded = base64.b64encode(video)
# # HTML(data='''<video alt="test" controls>
# #                 <source src="data:video/mp4;base64,{0}" type="video/mp4" />
# #              </video>'''.format(encoded.decode('ascii')))
# im = cv2.imread('training/unseen_speakers/test_pic.jpg')
# ims = get_frames_mouth(detector, predictor, [im])
# plt.imshow(ims[0][...,::-1])

In [ ]:
# # np.max([len(word) for word in words])
# type(np.array([1,2]))

In [ ]:
# lipnet.predict(np.array([X]))

In [ ]:
# m = []
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append(1)
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([[1],[1],[1]])
# np.array(m).shape
t = {1,2,3,4,5}
t.add(5)
t.add(6)

In [ ]:
t